# Multiprocessing in python

The **multiprocessing library** is Python's standard library to support parallel computing using processes.

See documentation here:

https://docs.python.org/3/library/multiprocessing.html

In [ ]:
import multiprocessing as mp

Let's first print out the total number of CPUs that on my machine that can be used for parallel computing.

In [ ]:
print(f"Number of CPU cores: {mp.cpu_count()}")

Let's use an example to show you how to use multiple cores in one machine to reduce the time of execution time. 

### Example 1:

Generate $10^7$ random numbers between 0 and 10, and square the number. Store the results in a list. 

#### A) Serial version:

In [ ]:
import numpy as np
import time
import random_square # our own module

In [ ]:
# Timing the code
start = time.time()

# Empty  list for results 
results = []

# Loop up to 10^7

for i in range(10000000): 

    results.append(random_square.random_square(i))

# Finish timing code
end =time.time()

print(f'Serial execution time is {end - start} s')

#### B) Parallel version

The simplest way to do parallel computing using the multiprocessing is to use the **Pool** class.

See documentation:

https://superfastpython.com/multiprocessing-pool-map/

https://docs.python.org/3/library/multiprocessing.html#multiprocessing.pool.Pool


There are 4 common methods in the class that we may use often, that is:

1. **apply**, 


2. **map**, 


3. **apply_async** and 


4. **map_async**.

Have a look of the documentation for the differences, and we will only use **map** function below to parallel the above example.

The **map(func, iterable)** function takes in two arguments, and apply the function **func** to each element in the **iterable**, and then collect the results. 

In [ ]:
# Timing the code
start = time.time()

# Get number of laptop CPUs
n_cpu = mp.cpu_count()

# Call Pool
pool = mp.Pool(processes=n_cpu)

# Call pool.map(func, iterable)
results = [pool.map(random_square.random_square, range(10000000))]

# Finish timing code
end = time.time()

print(f'Parallel execution time is {end - start} s')

#### Notes:


- We can see that using the above parallel version of the code, we reduce the run time from 59.18333172798157 s to 30.407769918441772 s. This is a big gain in speed, especially if we have a heavy computation, it will reduce a lot of time by running parallel computing. 


- The `pool.apply` function is similar except that it can accept more arguments. The `pool.map` and `pool.apply` will lock the main program until all the processes are finished, which is quite useful if we want to obtain results in a particular order for some applications.


- In contrast, if we do not need the results in a particular order, we can also use `pool.apply_async` or `pool.map_async`, which will submit all processes at once and retrieve the results as soon as they are finished. Check documentation to learn more. 

## Visualise the execution time 

Let's visualise the execution time changes versus the number of data points using both the serial and parallel version.

We will see that until certain point, it is better to use the serial version. 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Serial function

def serial(n):
    
    start = time.time()
    results = []
    
    for i in range(n): 
        results.append(random_square.random_square(i))
    
    end = time.time()
    exec_time = end - start
    
    return exec_time

# Parallel function

def parallel(n):
    
    start = time.time()
    n_cpu = mp.cpu_count()

    pool = mp.Pool(processes=n_cpu)
    
    results = [pool.map(random_square.random_square, range(n))]
    
    end = time.time()
    exec_time = end - start
    
    return exec_time

In [ ]:
# Generate numbers in log-space:
n_run = np.logspace(1, 7, num = 7)

print(n_run)

# Call functions for each n_run[i]

t_serial = [serial(int(n)) for n in n_run]

t_parallel = [parallel(int(n)) for n in n_run]

In [ ]:
print(t_serial)
print(t_parallel)

In [ ]:
plt.figure(figsize = (5, 4))

plt.plot(n_run, t_serial, '-o', color = "red", label = 'serial')
plt.plot(n_run, t_parallel, '-o', color = "green", label = 'parallel')

plt.loglog()
plt.legend()

plt.ylabel('Execution time (s)')
plt.xlabel('Number of random points')

plt.show()

### Conclusion:

We can see from the figure that when the number of data points are small (below 10000), the execution time for the serial version is faster due to the overheads of the parallel version from launching and maintaining the new processes. But after that, we can see clearly the winner be the parallel version. 

# Using joblib


In Python, there are also other 3rd party packages that can make the parallel computing easier, especially for some daily tasks.

**joblib** is one of them, it provides an easy simple way to do parallel computing (it has many other usages as well). 


#### See documentation:

https://joblib.readthedocs.io/en/latest/index.html

First you need to install it by running 

```python
conda install joblib
```

Or:

```python
pip install joblib
```

Let's see how can we run the previous example using this new package. 

In [1]:
from joblib import Parallel, delayed

import numpy as np
import time
import random_square

In [ ]:
# Time stamp at the beginning of the execution
start = time.time()

results = Parallel(n_jobs = 1)(delayed(random_square.random_square)(i) for i in range(1000000))

# Time stamp at the end of the execution
end = time.time()

# Print execution
print(end - start)

In [2]:
# Time stamp at the beginning of the execution
start = time.time()

results = Parallel(n_jobs = 2)(delayed(random_square.random_square)\
          (i) for i in range(1000000))

# Time stamp at the end of the execution
end = time.time()

# Print execution
print(end - start)

18.82672882080078


We can see the parallel part of the code becomes one line by using the joblib library, which is very convenient.


The **Parallel** is a helper class that essentially provides a convenient interface for the **multiprocessing** module we saw before.

The **delayed** is used to capture the arguments of the target function, in this case, the **random_square**. We run the above code with 2 cores.

### Notes on usage:

If you want to use all of the computational power on your machine. You can use all cores on your machine by setting *n\_jobs=-1*.


If you set it to -2, all CPUs but one are used. 


Besides, you can turn on the **verbose** argument to output the status messages. 

In [3]:
results = Parallel(n_jobs=-1, verbose=1)(delayed(random_square.random_square)(i) for i in range(1000000))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 360456 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done 1000000 out of 1000000 | elapsed:   20.1s finished


### Backends

There are multiple backends in **joblib**, which means using different ways to do the parallel computing.

If you set the backend as **multiprocessing**, this is actually creating, under the hood, a multiprocessing pool that uses separate Python woker processes to execute tasks concurrently on separate cores. 

In [4]:
results = Parallel(n_jobs=-1, backend='multiprocessing', verbose=1)\
          (delayed(random_square.random_square)(i) for i in range(1000000))

[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 312 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 786424 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done 1000000 out of 1000000 | elapsed:    6.6s finished
